<a href="https://colab.research.google.com/github/Sateesh110/Rep_ML/blob/master/zML_SL_Regress_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Before we Begin:

#Problem Statement :

#Solution Approach:

##Assumptions:

#Objective of this Kernel:

#Credits

#Import Packages

In [1]:
!pip install catboost

     |████████████████████████████████| 65.8MB 86kB/s 


In [2]:
# Data Processing
import numpy as np # linear algebra
import pandas as pd # data processing
from numpy import sort

# Visulaization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
from pandas.plotting import scatter_matrix

In [3]:
# XGBoost
import xgboost as xgb

# lightGBM
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier

# Scikit learn Classfiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor ,RandomForestRegressor,GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
#from sklearn.tree import DecisionTreeClassifier

#Catboost
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier

In [63]:
# Other Scikit Learn
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats import reciprocal, uniform
from scipy.stats import expon
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from scipy.stats import randint, uniform
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedShuffleSplit

In [15]:
# Others
import time
import collections
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import gc
import shutil

# Reproducibility
seed = 342
np.random.seed(seed)

#import image
from IPython.display import Image

import os
import tarfile
from six.moves import urllib

##Custom Functions

In [47]:
def df_stats(df,columns):
  """
  df info
  """
  stats = []
  for col in columns:
    stats.append((col, df[col].count(), df[col].nunique(), df[col].isnull().sum(), df[col].isnull().sum() * 100 / df.shape[0], df[col].dtype))
    stats_cols = ['attribute', 'record_count', 'unique_values','missing_records', 'percent_missing_records', 'data_type']
    df_stats1 = df.describe().T
    df_stats1.index.names = ['attribute']
    df_stats2 = pd.DataFrame(stats, columns=stats_cols)
    df_stats2.set_index('attribute', inplace=True)
    df_stats = pd.concat(([df_stats2,df_stats1]),axis=1)
  return(df_stats)

In [7]:
def drop_cols(df,cols):
  """
  drop columns which have missing records > 90%
  """
  drop_cols = []
  n = 90
  for col in cols:
    rate = (df[col].isnull().sum() * 100 / df.shape[0])
    if rate > n:
      df = df.drop(col, axis=1)
      drop_cols.append(col)
  return(df,drop_cols)

In [8]:
def CalcOutliers(df_num):
  # calculating mean and std of the array
  data_mean, data_std = np.mean(df_num), np.std(df_num)
  # seting the cut line to both higher and lower values
  # You can change this value
  cut = data_std * 3
  #Calculating the higher and lower cut values
  lower, upper = data_mean - cut, data_mean + cut
  # creating an array of lower, higher and total outlier values
  outliers_lower = [x for x in df_num if x < lower]
  outliers_higher = [x for x in df_num if x > upper]
  outliers_total = [x for x in df_num if x < lower or x > upper]
  # array without outlier values
  outliers_removed = [x for x in df_num if x > lower and x < upper]
  print('Identified lowest outliers: %d' % len(outliers_lower)) # printing total number of values in lower cut of outliers
  print('Identified upper outliers: %d' % len(outliers_higher)) # printing total number of values in higher cut of outliers
  print('Total outlier observations: %d' % len(outliers_total)) # printing total number of values outliers of both sides
  print('Non-outlier observations: %d' % len(outliers_removed)) # printing total number of non outlier values
  print("Total percentual of Outliers: ", round((len(outliers_total) / len(outliers_removed) )*100, 4)) # Percentual of outliers in points
  return

In [9]:
def auc_score(y_true, y_pred):
  """
  Calculates the Area Under ROC Curve (AUC)
  """
  return roc_auc_score(y_true, y_pred)

In [10]:
def plot_curve(y_true_train, y_pred_train, y_true_val, y_pred_val, model_name):
  """
  Plots the ROC Curve given predictions and labels
  """
  fpr_train, tpr_train, _ = roc_curve(y_true_train, y_pred_train, pos_label=1)
  fpr_val, tpr_val, _ = roc_curve(y_true_val, y_pred_val, pos_label=1)
  plt.figure(figsize=(8, 8))
  plt.plot(fpr_train, tpr_train, color='black',
  lw=2, label=f"ROC train curve (AUC = {round(roc_auc_score(y_true_train, y_pred_train), 4)})")
  plt.plot(fpr_val, tpr_val, color='darkorange',lw=2, label=f"ROC validation curve (AUC = {round(roc_auc_score(y_true_val, y_pred_val), 4)})")
  plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  plt.xlim([-0.05, 1.05])
  plt.ylim([-0.05, 1.05])
  plt.xlabel('False Positive Rate', fontsize=16)
  plt.ylabel('True Positive Rate', fontsize=16)
  plt.xticks(fontsize=14)
  plt.yticks(fontsize=14)
  plt.title(f'ROC Plot for {model_name}', weight="bold", fontsize=20)
  plt.legend(loc="lower right", fontsize=16)

In [11]:
def reduce_mem_usage(df):
  """
  Reduces memory usage for all columns in a Pandas DataFrame
  """
  start_mem_usg = df.memory_usage().sum() / 1024**2
  print("Memory usage of properties dataframe is :",start_mem_usg," MB")
  NAlist = [] # Keeps track of columns that have missing values filled in.
  for col in df.columns:
    if df[col].dtype != object: # Exclude strings # make variables for Int, max and min
      IsInt = False
      mx = df[col].max()
      mn = df[col].min()
    # Integer does not support NA, therefore, NA needs to be filled
    if not np.isfinite(df[col]).all():
      NAlist.append(col)
      df[col].fillna(mn-1,inplace=True)
  # test if column can be converted to an integer
  asint = df[col].fillna(0).astype(np.int32)
  result = (df[col] - asint)
  result = result.sum()
  if result > -0.01 and result < 0.01:
    IsInt = True
  # Make Integer/unsigned Integer datatypes
  if IsInt:
    if mn >= 0:
      if mx < 255:
        df[col] = df[col].astype(np.uint8)
      elif mx < 65535:
        df[col] = df[col].astype(np.uint16)
      else:
        df[col] = df[col].astype(np.uint32)
    else:
      if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
        df[col] = df[col].astype(np.int8)
      elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
        df[col] = df[col].astype(np.int16)
      else:
        df[col] = df[col].astype(np.int32)
  # Make float datatypes 32 bit
  else:
    df[col] = df[col].astype(np.float32)
    
  # Print final result
  mem_usg = df.memory_usage().sum() / 1024**2
  print("Memory usage of properties dataframe is after reduction is:",mem_usg," MB")
  return df, NAlist

In [12]:
def plot_predictions(regressors, X, y, axes, label=None, style="r-", data_style="b.", data_label=None):
  x1 = np.linspace(axes[0], axes[1], 500)
  y_pred = sum(regressor.predict(x1.reshape(-1, 1)) for regressor in regressors)
  plt.plot(X[:, 0], y, data_style, label=data_label)
  plt.plot(x1, y_pred, style, linewidth=2, label=label)
  if label or data_label:
    plt.legend(loc="upper center", fontsize=16)
    plt.axis(axes)

In [56]:
# Where to save the figures
PROJECT_ROOT_DIR = "/content/datasets"
CHAPTER_ID = "Housing"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

#Data Ingestion

##GitHub Source

In [22]:
#Initialize variables

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

#Get the data from Github to local drive
#Unzip the csv file

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

#Read using Pandas

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [23]:
# Get the data
fetch_housing_data()
housing = load_housing_data()
housing.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


In [49]:
cols = housing.columns
df_stats(housing,cols)

,record_count,unique_values,missing_records,percent_missing_records,data_type,count,mean,std,min,25%,50%,75%,max
longitude,20640,844,0,0.000000,float64,20640.0,-119.569704,2.003532,-124.3500,-121.8000,-118.4900,-118.01000,-114.3100
latitude,20640,862,0,0.000000,float64,20640.0,35.631861,2.135952,32.5400,33.9300,34.2600,37.71000,41.9500
housing_median_age,20640,52,0,0.000000,float64,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
total_rooms,20640,5926,0,0.000000,float64,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
total_bedrooms,20433,1923,207,1.002907,float64,20433.0,537.870553,421.385070,1.0000,296.0000,435.0000,647.00000,6445.0000
population,20640,3888,0,0.000000,float64,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640,1815,0,0.000000,float64,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
median_income,20640,12928,0,0.000000,float64,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
median_house_value,20640,3842,0,0.000000,float64,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000
ocean_proximity,20640,5,0,0.000000,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Data Preparation

#Exploratory Analysis

In [13]:
%matplotlib inline
housing.hist(bins=50, figsize=(20,15))
#save_fig("attribute_histogram_plots")
plt.show()

# Feature Engineering

# Data Split: Train Vs Test

In [64]:
# Random Split 80:20
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
print("Test records",len(test_set))
print("Train records",len(train_set))

Test records 4128
Train records 16512


In [66]:
#Random. But, Stratified Split 80:20

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["ocean_proximity"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [68]:
strat_test_set["ocean_proximity"].value_counts() / len(strat_test_set)

<1H OCEAN     0.442587
INLAND        0.317345
NEAR OCEAN    0.128876
NEAR BAY      0.110950
ISLAND        0.000242
Name: ocean_proximity, dtype: float64

In [69]:
strat_train_set["ocean_proximity"].value_counts() / len(strat_test_set)

<1H OCEAN     1.770591
INLAND        1.269622
NEAR OCEAN    0.515019
NEAR BAY      0.443798
ISLAND        0.000969
Name: ocean_proximity, dtype: float64

#Model Selection & Validation

#Model Comparison & Metrics

#Model Testing

#Summary